Imports & App Setup

In [33]:
# Cell 1: Imports and App Setup
import customtkinter as ctk
import tkinter.filedialog as fd
import os

ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("blue")

app = ctk.CTk()
app.title("Smart File Upload System")
app.geometry("1200x700")

# Global file lists
file_list = []
submitted_files = []


Sidebar UI

In [34]:
# Cell 2: Sidebar creation
sidebar = ctk.CTkFrame(app, width=150)
sidebar.pack(side="left", fill="y")

file_button = ctk.CTkButton(sidebar, text="File")
file_button.pack(pady=10)

search_button = ctk.CTkButton(sidebar, text="Search")
search_button.pack(pady=10)

collapse_button = ctk.CTkButton(sidebar, text="Collapse")
collapse_button.pack(side="bottom", pady=10)


Main Frame and File List UI

In [35]:
# Cell 3: Main frame and file add frame setup
main_frame = ctk.CTkFrame(app)
main_frame.pack(side="left", fill="both", expand=True)

file_add_frame = ctk.CTkFrame(main_frame)
file_add_frame.pack(padx=10, pady=(10, 0), fill="x")


File Adding Logic and Button


In [36]:
# Cell 4: File listbox display and Add File/Folder buttons

file_listbox = ctk.CTkTextbox(file_add_frame, height=60, width=600)
file_listbox.pack(side="left", fill="x", expand=True)
file_listbox.configure(state="disabled")

def add_file():
    path = fd.askopenfilename()
    if path and path not in file_list:
        file_list.append(path)
        file_listbox.configure(state="normal")
        file_listbox.insert("end", os.path.basename(path) + "\n")
        file_listbox.configure(state="disabled")

add_file_button = ctk.CTkButton(file_add_frame, text="Add File", command=add_file, width=100)
add_file_button.pack(side="left", padx=(10, 5))

def add_folder():
    folder_path = fd.askdirectory()
    if folder_path:
        for root, _, files in os.walk(folder_path):
            for file in files:
                full_path = os.path.join(root, file)
                if full_path not in file_list:
                    file_list.append(full_path)
                    file_listbox.configure(state="normal")
                    relative_path = os.path.relpath(full_path, folder_path)
                    file_listbox.insert("end", f"{os.path.basename(folder_path)}/{relative_path}\n")
                    file_listbox.configure(state="disabled")

add_folder_button = ctk.CTkButton(file_add_frame, text="Add Folder", command=add_folder, width=100)
add_folder_button.pack(side="left")


terminal display

In [37]:
# Cell 5: Terminal textbox for log display
terminal = ctk.CTkTextbox(app)
terminal.pack(side="right", fill="both", expand=True, padx=10, pady=10)
terminal.configure(state="disabled")


Dropdown and Submit Logic

In [38]:
# Cell 6: Submit button and submit logic

def submit_files():
    global submitted_files
    submitted_files = file_list.copy()
    update_dropdown()

    terminal.configure(state="normal")
    terminal.delete("1.0", "end")
    folder_map = {}

    for path in submitted_files:
        folder = os.path.dirname(path)
        folder_map.setdefault(folder, []).append(path)

    for folder, files in folder_map.items():
        folder_name = os.path.basename(folder)
        if len(files) > 1:
            terminal.insert("end", f"Analyzing {folder_name}...\n")
        for file_path in files:
            relative = os.path.relpath(file_path, folder)
            terminal.insert("end", f"Analyzing {folder_name}/{relative}\n")

    terminal.configure(state="disabled")

    file_listbox.configure(state="normal")
    file_listbox.delete("1.0", "end")
    file_listbox.configure(state="disabled")

    file_list.clear()

submit_button = ctk.CTkButton(main_frame, text="Submit", command=submit_files)
submit_button.pack(padx=10, pady=(10, 0), fill="x")


dropdown UI

In [39]:
# Cell 7: Dropdown for submitted files and update/select handlers

selected_file = ctk.StringVar()

def update_dropdown():
    menu = [os.path.basename(f) for f in submitted_files]
    dropdown.configure(values=menu)
    if menu:
        dropdown.set(menu[0])

def on_file_select(choice):
    summary_box.configure(state="normal")
    summary_box.delete("1.0", "end")
    summary_box.insert("end", f"AI Summary of {choice}")
    summary_box.configure(state="disabled")

dropdown = ctk.CTkOptionMenu(main_frame, variable=selected_file, command=on_file_select)
dropdown.pack(padx=10, pady=(10, 0), fill="x")
dropdown.set("No file submitted yet")


Summary & Chat UI

In [40]:
# Cell 8: Summary and chat section layout

summary_chat_frame = ctk.CTkFrame(main_frame)
summary_chat_frame.pack(padx=10, pady=(10, 0), fill="both", expand=True)

summary_box = ctk.CTkTextbox(summary_chat_frame, height=100)
summary_box.pack(padx=5, pady=(5, 0), fill="x")
summary_box.configure(state="disabled")

save_button = ctk.CTkButton(summary_chat_frame, text="Save")
save_button.pack(anchor="ne", padx=5, pady=5)

chat_box = ctk.CTkTextbox(summary_chat_frame)
chat_box.pack(padx=5, pady=(0, 5), fill="both", expand=True)
chat_box.configure(state="disabled")


Chat Logic

In [41]:
# Cell 9: Chat input area and send chat logic

chat_input_frame = ctk.CTkFrame(summary_chat_frame)
chat_input_frame.pack(fill="x", pady=(0, 10), padx=5)

chat_input = ctk.CTkEntry(chat_input_frame)
chat_input.pack(side="left", fill="x", expand=True, padx=(0, 10))

def send_chat():
    user_msg = chat_input.get()
    if user_msg:
        chat_box.configure(state="normal")
        chat_box.insert("end", f"You: {user_msg}\n")
        chat_box.insert("end", "AI: This is a sample response\n")
        chat_box.configure(state="disabled")
        chat_input.delete(0, "end")

send_button = ctk.CTkButton(chat_input_frame, text="Send", command=send_chat)
send_button.pack(side="right")


Run the app

In [42]:
# Cell 10: Run the app main loop
app.mainloop()


KeyboardInterrupt: 